<a href="https://colab.research.google.com/github/syedhammadahmed/quantumcomputings20/blob/master/hw_3/quantum_computing_hw_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## HW 3

### Problem 1 

Let $\omega=\mathrm{exp}(2\pi i/N)$ be the $N$th root of unity.  The Fourier matrix of size $N$ is the matrix $F_N=(q_{k,\ell})\in\mathbb{C}^{N\times N}$ whose entries are given by
$$q_{k,\ell}=\frac{1}{\sqrt{N}} \omega^{k \cdot \ell}$$ for $k, \ell \in \{0,\ldots,N-1\}$.

Using numpy, implement the function ```get_fourier_matrix``` that create the Fourier matrix of size $N$.

Compute the eigenvalues of eigenvectors of the Fourier matrix.  

To check that your code is correct, you should verify that the Fourier matrix is unitary and that its eigenvalues are $N$th roots of unity.

In [0]:
import math
import numpy as np
import numpy.linalg as NPLA
import scipy.linalg as SPLA

# author: Syed Hammad Ahmed (5068474)

# to print values to this significant figures
roundTo = 4

# returns the n-th root of unity power k*l (Fourier matrix element [k][l])
def getNthRootMatrixValue(k, l, n):
  real = math.cos(2 * (k * l) * math.pi * 1.0/n)
  im = math.sin(2 * (k * l) * math.pi * 1.0/n)
  return (1.0/math.sqrt(n))*complex(real, im)

# returns fourier matrix where n is the argument
# n is the root value for n-th root of unity
def get_fourier_matrix(n):
  fourierMatrix = np.zeros((n,n), dtype=complex)
  for k in range(0, n):
    for l in range(0, n): 
      fourierMatrix[k][l] = getNthRootMatrixValue(k, l, n)
  return fourierMatrix

# returns the eigen values of a matrix
def getEigenValNVec(matrix):
  return NPLA.eigvals(matrix)

# returns the dagger (complex conjugate) of a matrix
def getDagger(matrix):
  return np.conj(matrix)

# returns the inverse of a matrix
def getInverse(matrix):
  return NPLA.inv(matrix)
  
#test driver code tests for values of n = 4 to 1
def main():
  i = 4
  while(i):
    print("n =",i)
    fourierMatrix = get_fourier_matrix(i)
    print("Fourier Matrix:")
    print(np.round(fourierMatrix, roundTo))
    print("Dagger (Complex Conjugate):")
    print(np.round(getDagger(fourierMatrix), roundTo))
    print("Inverse:")
    print(np.round(getInverse(fourierMatrix), roundTo))
    print("Dagger == Inverse, Hence, fourier matrix is unitary!")

    eigenValues = np.round(getEigenValNVec(fourierMatrix), roundTo)
    print("eigenValues:", eigenValues)
    print("eigenValues^n:", np.power(eigenValues, i))
    print("Each eigen value power n == 1. Hence, it is n-th root of unity!")

    i = i - 1
    print("\n"*2)


if __name__ == "__main__":
    main()

n = 4
Fourier Matrix:
[[ 0.5+0.j   0.5+0.j   0.5+0.j   0.5+0.j ]
 [ 0.5+0.j   0. +0.5j -0.5+0.j  -0. -0.5j]
 [ 0.5+0.j  -0.5+0.j   0.5-0.j  -0.5+0.j ]
 [ 0.5+0.j  -0. -0.5j -0.5+0.j   0. +0.5j]]
Dagger (Complex Conjugate):
[[ 0.5-0.j   0.5-0.j   0.5-0.j   0.5-0.j ]
 [ 0.5-0.j   0. -0.5j -0.5-0.j  -0. +0.5j]
 [ 0.5-0.j  -0.5-0.j   0.5+0.j  -0.5-0.j ]
 [ 0.5-0.j  -0. +0.5j -0.5-0.j   0. -0.5j]]
Inverse:
[[ 0.5-0.j   0.5-0.j   0.5-0.j   0.5+0.j ]
 [ 0.5+0.j   0. -0.5j -0.5+0.j  -0. +0.5j]
 [ 0.5+0.j  -0.5-0.j   0.5+0.j  -0.5-0.j ]
 [ 0.5+0.j  -0. +0.5j -0.5-0.j   0. -0.5j]]
Dagger == Inverse, Hence, fourier matrix is unitary!
eigenValues: [ 1.+0.j -1.+0.j  0.+1.j  1.-0.j]
eigenValues^n: [1.+0.j 1.+0.j 1.+0.j 1.+0.j]
Each eigen value power n == 1. Hence, it is n-th root of unity!



n = 3
Fourier Matrix:
[[ 0.5774+0.j   0.5774+0.j   0.5774+0.j ]
 [ 0.5774+0.j  -0.2887+0.5j -0.2887-0.5j]
 [ 0.5774+0.j  -0.2887-0.5j -0.2887+0.5j]]
Dagger (Complex Conjugate):
[[ 0.5774-0.j   0.5774-0.j   0.57

### Problem 2

Using Qiskit IBM Q Experience, implement the quantum phase estimation circuit with 3-bit precision.  

This quantum circuit is explained on pages 110-113 (Inverse Fourier transform for 3 qubits) of the lecture slides.

You can preprare the initial state using ```u3``` gates. See [u3 gates](https://qiskit.org/documentation/tutorials/fundamentals/5_summary_of_quantum_operations.html?highlight=arbitrary%20single%20qubit%20gate#u-gates)

The controlled $R_2^\dagger$ gates are controlled ```u3``` gates.  See [controlled u3 gate](https://qiskit.org/documentation/tutorials/fundamentals/5_summary_of_quantum_operations.html?highlight=arbitrary%20single%20qubit%20gate#Controlled-u3-rotation).

In [1]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, execute
from qiskit.tools.visualization import circuit_drawer
from qiskit.quantum_info import state_fidelity
from qiskit import BasicAer
# Useful additional packages
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from math import pi

backend = BasicAer.get_backend('unitary_simulator')

q = QuantumRegister(3)
c = ClassicalRegister(3)

qc = QuantumCircuit(q, c)
qc.h(q[0])
qc.cu3(pi/2, pi/2, pi/2, q[0], q[1])
qc.h(q[1])
qc.cu3(pi/2, pi/2, pi/2, q[0], q[2])
qc.cu3(pi/2, pi/2, pi/2, q[1], q[2])
qc.h(q[2])
qc.measure(q, c)
qc.draw()


┌───┐                                                 »
q0_0: ┤ H ├──────────■──────────────────────────■───────────»
      └───┘┌─────────┴──────────┐┌───┐          │           »
q0_1: ─────┤ U3(pi/2,pi/2,pi/2) ├┤ H ├──────────┼───────────»
           └────────────────────┘└───┘┌─────────┴──────────┐»
q0_2: ────────────────────────────────┤ U3(pi/2,pi/2,pi/2) ├»
                                      └────────────────────┘»
c0_0: ══════════════════════════════════════════════════════»
                                                            »
c0_1: ══════════════════════════════════════════════════════»
                                                            »
c0_2: ══════════════════════════════════════════════════════»
                                                            »
«                            ┌─┐           
«q0_0: ──────────────────────┤M├───────────
«                            └╥┘     ┌─┐   
«q0_1: ──────────■────────────╫──────┤M├───
«      ┌─────────┴──────────┐ ║ ┌───┐└╥┘┌─┐
«q0_2: ┤ U3(pi/2,pi/2,pi/2) ├─╫─┤ H ├─╫─┤M├
«      └────────────────────┘ ║ └───┘ ║ └╥┘
«c0_0: ═══════════════════════╩═══════╬══╬═
«                                     ║  ║ 
«c0_1: ═══════════════════════════════╩══╬═
«                                        ║ 
«c0_2: ══════════════════════════════════╩═
«